In [5]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import cv2
import torch
import numpy as np
from torchvision import transforms

ModuleNotFoundError: No module named 'cv2'

In [2]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
device=torch.cuda.get_device_name()
device

'NVIDIA RTX 2000 Ada Generation Laptop GPU'